In [2]:
import pandas as pd
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm
from qdrant_client.models import Distance, VectorParams, PointStruct, TextIndexParams, TokenizerType
import os
import torch
import uuid


c:\Users\dczqd\miniconda3\envs\webapp\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

True


device(type='cuda')

In [5]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
model = model.to(device)

client = QdrantClient(url="http://localhost:6333")

# if (client.collection_exists(collection_name="amazon_products")):
#     client.delete_collection(collection_name="amazon_products")
    
collection_name = "amazon_products"

In [6]:

df_products = pd.read_csv(r'C:\Users\dczqd\Documents\test_product_reco\data\archive\amazon_products.csv')
df_category = pd.read_csv(r'C:\Users\dczqd\Documents\test_product_reco\data\archive\amazon_categories.csv')
df_combined = pd.merge(df_products, df_category, left_on='category_id', right_on='id')
df_combined = df_combined.drop(columns=['id', 'category_id', 'productURL']).dropna()
df_combined

,asin,title,imgUrl,stars,reviews,price,listPrice,isBestSeller,boughtInLastMonth,category_name
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,4.5,0,139.99,0.00,False,2000,Suitcases
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,4.5,0,169.99,209.99,False,1000,Suitcases
2,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,https://m.media-amazon.com/images/I/71EA35zvJB...,4.6,0,365.49,429.99,False,300,Suitcases
3,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,https://m.media-amazon.com/images/I/91k6NYLQyI...,4.6,0,291.59,354.37,False,400,Suitcases
4,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,https://m.media-amazon.com/images/I/61NJoaZcP9...,4.5,0,174.99,309.99,False,400,Suitcases
...,...,...,...,...,...,...,...,...,...,...
1426332,B00R3LIKCO,American Flag Patriotic USA Classic 5 Panel Me...,https://m.media-amazon.com/images/I/71PDJFz6AA...,4.2,0,14.95,0.00,False,0,Men's Accessories
1426333,B098BQ7ZQ3,Men's Baseball Cap - H2O-DRI Line Up Curved Br...,https://m.media-amazon.com/images/I/812Tycexs4...,4.4,0,33.99,0.00,False,0,Men's Accessories
1426334,B07X1MVNT1,[4 Pack] Adjustable Eyeglasses and Sunglasses ...,https://m.media-amazon.com/images/I/61vvYW1S9J...,3.6,0,8.54,0.00,False,0,Men's Accessories
1426335,B08XLBG8V9,Ax2002 Aviator Sunglasses,https://m.media-amazon.com/images/I/51+yjD4F1x...,4.5,0,54.36,57.39,False,0,Men's Accessories


In [7]:
df_combined['combined_features'] = df_combined.apply(lambda x: f"{x['title']} {x['category_name']}".lower(), axis=1)
df_combined_small =     df_combined.sample(n=200000, random_state=42)
df_combined_small.to_csv('df_combined_small.csv', index=False)
df_combined_small


,asin,title,imgUrl,stars,reviews,price,listPrice,isBestSeller,boughtInLastMonth,category_name,combined_features
1208301,B0BKZNNL1V,"LOVEVOOK Diaper Bag Backpack, Diaper Backpack ...",https://m.media-amazon.com/images/I/718pt0oLES...,4.6,0,36.99,0.00,False,0,Baby Travel Gear,"lovevook diaper bag backpack, diaper backpack ..."
1089181,B0968QPTK8,Seloom 148 PCS Reflective Mailbox Numbers and ...,https://m.media-amazon.com/images/I/71Sr2+igJf...,4.5,286,14.89,19.99,False,50,Hardware,seloom 148 pcs reflective mailbox numbers and ...
466837,B00NIAULVC,"Spry Xylitol Toothpaste 5oz, Fluoride Toothpas...",https://m.media-amazon.com/images/I/51I687iehe...,4.7,0,8.49,8.99,False,900,Oral Care Products,"spry xylitol toothpaste 5oz, fluoride toothpas..."
39647,B0794ZFTDB,2 Pieces Rhinestone Bow Ties Banquet Bowties M...,https://m.media-amazon.com/images/I/81G1Dc1uZu...,4.4,0,12.99,13.99,False,50,Men's Accessories,2 pieces rhinestone bow ties banquet bowties m...
485619,B075PTHVKN,Mielle Organics Pomegranate & Honey Leave-In C...,https://m.media-amazon.com/images/I/71RNqb6TfJ...,4.7,0,12.52,14.99,False,10000,Hair Care Products,mielle organics pomegranate & honey leave-in c...
...,...,...,...,...,...,...,...,...,...,...,...
235600,B09PNR1DD2,PAINTS TRANSFER Chalk Stencils Transfers Mesh ...,https://m.media-amazon.com/images/I/51S8Jo7Aa5...,4.8,18,9.59,11.99,False,0,Printmaking Supplies,paints transfer chalk stencils transfers mesh ...
147966,B0BBMCPD31,LumStory 12-Pack GU10 LED Bulbs Daylight White...,https://m.media-amazon.com/images/I/61MjC05HeH...,4.4,186,23.99,0.00,False,0,Light Bulbs,lumstory 12-pack gu10 led bulbs daylight white...
717910,B00R9NUL3W,BSC-N01 Pneumatic Speed Control Silencer Valve...,https://m.media-amazon.com/images/I/61LSignWbq...,5.0,0,5.66,0.00,False,0,Filtration,bsc-n01 pneumatic speed control silencer valve...
714824,B0BGGRJ1WJ,Water Softener Venturi Gasket for Whirlpool Ke...,https://m.media-amazon.com/images/I/51wtlMMPGn...,4.2,0,10.70,0.00,False,0,Filtration,water softener venturi gasket for whirlpool ke...


In [8]:
batch_size = 512
embeddings = []
def encode_batch(model, batch, device):
    return model.encode(batch, device=device)

for i in tqdm(range(0, len(df_combined_small), batch_size), desc="Encoding rows"):
    batch = df_combined_small['combined_features'].iloc[i:i + batch_size].tolist()
    batch_embeddings = encode_batch(model, batch, device)
    embeddings.extend(batch_embeddings)

# Convert the list of embeddings to a DataFrame
df_combined_small['embeddings'] = embeddings



Encoding rows:   0%|          | 0/391 [00:00<?, ?it/s]c:\Users\dczqd\miniconda3\envs\webapp\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Encoding rows: 100%|██████████| 391/391 [01:03<00:00,  6.13it/s]


In [9]:
df_combined_small.head()

,asin,title,imgUrl,stars,reviews,price,listPrice,isBestSeller,boughtInLastMonth,category_name,combined_features,embeddings
1208301,B0BKZNNL1V,"LOVEVOOK Diaper Bag Backpack, Diaper Backpack ...",https://m.media-amazon.com/images/I/718pt0oLES...,4.6,0,36.99,0.00,False,0,Baby Travel Gear,"lovevook diaper bag backpack, diaper backpack ...","[-0.022773411, 0.05414636, 0.027121775, 0.0104..."
1089181,B0968QPTK8,Seloom 148 PCS Reflective Mailbox Numbers and ...,https://m.media-amazon.com/images/I/71Sr2+igJf...,4.5,286,14.89,19.99,False,50,Hardware,seloom 148 pcs reflective mailbox numbers and ...,"[-0.0035544634, -0.02253524, 0.030703753, -0.0..."
466837,B00NIAULVC,"Spry Xylitol Toothpaste 5oz, Fluoride Toothpas...",https://m.media-amazon.com/images/I/51I687iehe...,4.7,0,8.49,8.99,False,900,Oral Care Products,"spry xylitol toothpaste 5oz, fluoride toothpas...","[-0.0131346, -0.12989144, 0.063452646, 0.00754..."
39647,B0794ZFTDB,2 Pieces Rhinestone Bow Ties Banquet Bowties M...,https://m.media-amazon.com/images/I/81G1Dc1uZu...,4.4,0,12.99,13.99,False,50,Men's Accessories,2 pieces rhinestone bow ties banquet bowties m...,"[0.005080655, 0.02874856, 0.015073913, 0.04693..."
485619,B075PTHVKN,Mielle Organics Pomegranate & Honey Leave-In C...,https://m.media-amazon.com/images/I/71RNqb6TfJ...,4.7,0,12.52,14.99,False,10000,Hair Care Products,mielle organics pomegranate & honey leave-in c...,"[-0.08184215, -0.017593145, 0.077282645, -0.01..."


In [10]:
def hash_to_uuid(unique_string):
    return str(uuid.uuid5(uuid.NAMESPACE_DNS, unique_string))
def create_point(row):
    return PointStruct(
        id=hash_to_uuid(row['asin']),
        vector=np.array(row['embeddings']).tolist(), 
        payload={
            'asin' : row['asin'],
            'title' : row['title'],
            'imgUrl' : row['imgUrl'], 
            'stars' : float(row['stars']),  
            'reviews' : int(row['reviews']),
            'price' : float(row['price']), 
            'category_name': str(row['category_name']),
         
        }
    )

points = []
for index, row in tqdm(df_combined_small.iterrows()):   
    point = create_point(row)
    points.append(point)
# try
# tqdm.pandas(desc="Processing rows")
# points = df_combined_small.progress_apply(create_point_from_row, axis=1).tolist()
print(len(points))


200000it [00:15, 12677.38it/s]

200000


In [11]:
response = client.create_collection(
    collection_name=f"{collection_name}",
    vectors_config=VectorParams(size=len(df_combined_small['embeddings'].tolist()[0]), distance=Distance.COSINE),
)
print(response)
client.get_collection(collection_name)

True


CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=0, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quant

In [12]:
print('dimensions',len(df_combined_small['embeddings'].tolist()[0]))

dimensions 384


In [13]:
df_combined_small

,asin,title,imgUrl,stars,reviews,price,listPrice,isBestSeller,boughtInLastMonth,category_name,combined_features,embeddings
1208301,B0BKZNNL1V,"LOVEVOOK Diaper Bag Backpack, Diaper Backpack ...",https://m.media-amazon.com/images/I/718pt0oLES...,4.6,0,36.99,0.00,False,0,Baby Travel Gear,"lovevook diaper bag backpack, diaper backpack ...","[-0.022773411, 0.05414636, 0.027121775, 0.0104..."
1089181,B0968QPTK8,Seloom 148 PCS Reflective Mailbox Numbers and ...,https://m.media-amazon.com/images/I/71Sr2+igJf...,4.5,286,14.89,19.99,False,50,Hardware,seloom 148 pcs reflective mailbox numbers and ...,"[-0.0035544634, -0.02253524, 0.030703753, -0.0..."
466837,B00NIAULVC,"Spry Xylitol Toothpaste 5oz, Fluoride Toothpas...",https://m.media-amazon.com/images/I/51I687iehe...,4.7,0,8.49,8.99,False,900,Oral Care Products,"spry xylitol toothpaste 5oz, fluoride toothpas...","[-0.0131346, -0.12989144, 0.063452646, 0.00754..."
39647,B0794ZFTDB,2 Pieces Rhinestone Bow Ties Banquet Bowties M...,https://m.media-amazon.com/images/I/81G1Dc1uZu...,4.4,0,12.99,13.99,False,50,Men's Accessories,2 pieces rhinestone bow ties banquet bowties m...,"[0.005080655, 0.02874856, 0.015073913, 0.04693..."
485619,B075PTHVKN,Mielle Organics Pomegranate & Honey Leave-In C...,https://m.media-amazon.com/images/I/71RNqb6TfJ...,4.7,0,12.52,14.99,False,10000,Hair Care Products,mielle organics pomegranate & honey leave-in c...,"[-0.08184215, -0.017593145, 0.077282645, -0.01..."
...,...,...,...,...,...,...,...,...,...,...,...,...
235600,B09PNR1DD2,PAINTS TRANSFER Chalk Stencils Transfers Mesh ...,https://m.media-amazon.com/images/I/51S8Jo7Aa5...,4.8,18,9.59,11.99,False,0,Printmaking Supplies,paints transfer chalk stencils transfers mesh ...,"[-0.035437427, -0.030382615, 0.01301161, -0.04..."
147966,B0BBMCPD31,LumStory 12-Pack GU10 LED Bulbs Daylight White...,https://m.media-amazon.com/images/I/61MjC05HeH...,4.4,186,23.99,0.00,False,0,Light Bulbs,lumstory 12-pack gu10 led bulbs daylight white...,"[-0.003312654, 0.033621773, 0.021981535, 0.020..."
717910,B00R9NUL3W,BSC-N01 Pneumatic Speed Control Silencer Valve...,https://m.media-amazon.com/images/I/61LSignWbq...,5.0,0,5.66,0.00,False,0,Filtration,bsc-n01 pneumatic speed control silencer valve...,"[-0.12660316, 0.05316353, 0.016821817, 0.01860..."
714824,B0BGGRJ1WJ,Water Softener Venturi Gasket for Whirlpool Ke...,https://m.media-amazon.com/images/I/51wtlMMPGn...,4.2,0,10.70,0.00,False,0,Filtration,water softener venturi gasket for whirlpool ke...,"[-0.07019925, 0.097576916, 0.0064715007, -0.06..."


In [14]:
print(points[0])

id='521cb15b-1909-5fa5-9990-20af8dfece28' vector=[-0.022773411124944687, 0.054146360605955124, 0.02712177485227585, 0.010426356457173824, 0.04270430654287338, 0.019809940829873085, 0.0754627212882042, 0.03705592453479767, -0.002641024300828576, 0.047447703778743744, 0.029203863814473152, -0.08328420668840408, 0.006050855852663517, -0.01911861263215542, 0.057172294706106186, 0.011903424747288227, 0.0016740531427785754, 0.01578545942902565, -0.033567674458026886, -0.03302691504359245, -0.055931735783815384, -0.003879731986671686, 0.052418921142816544, 0.029665157198905945, -0.06692888587713242, 0.12651486694812775, 0.010602708905935287, -0.03532126173377037, 0.012395143508911133, -0.06221972405910492, 0.026560010388493538, -0.07563735544681549, 0.027829090133309364, -0.04086369648575783, -0.02685016766190529, 0.011300060898065567, 0.0739482119679451, -0.019173966720700264, -0.011864908039569855, -0.018687423318624496, 0.03926179185509682, -0.00981119368225336, -0.031015336513519287, 0.03

In [15]:

def upload_batches(points, batch_size=100):

    total_batches = len(points) // batch_size + (1 if len(points) % batch_size > 0 else 0)

    for i in tqdm(range(total_batches), desc="Uploading batches"):
        start_index = i * batch_size
        end_index = start_index + batch_size
        batch_points = points[start_index:end_index]
 
        response = client.upsert(
            collection_name=f"{collection_name}",
            points=batch_points
        )

    

In [16]:
upload_batches(points, batch_size=400)


Uploading batches: 100%|██████████| 500/500 [02:43<00:00,  3.06it/s]


In [26]:

client.create_payload_index(
    collection_name=f"{collection_name}",
    field_name="title",
    field_schema=TextIndexParams(
        type="text",
        tokenizer=TokenizerType.WORD,
        min_token_len=1,
        max_token_len=20,
        lowercase=True,
    ),
)


UpdateResult(operation_id=517, status=<UpdateStatus.COMPLETED: 'completed'>)

In [30]:
client.create_payload_index(
    collection_name=f"{collection_name}",
    field_name="category_name",
    field_schema=TextIndexParams(
        type="text",
        tokenizer=TokenizerType.WORD,
        min_token_len=1,
        max_token_len=10,
        lowercase=True,
    ),
)

UpdateResult(operation_id=525, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
# results = client.search(
#         collection_name="amazon_products",
#         query_vector=query_vector.tolist(),
#         limit=n + (len(exclude_ids) if exclude_ids else 0),
#         query_filter=Filter(
#             must=filters
#                 )
#         )